<a href="https://colab.research.google.com/github/Shuji-Ishikawa/projects/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(['France','Germany', 'Canada', 'Italy', 'Japan', 'United Kingdom', 'United States'])][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['Capital Share'] = 1 - data['labsh']  # Capital share
data['TFP Share'] = data['labsh']

# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['Growth Rate'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita

data['TFPgrowth'] = data['Growth Rate'] * data['TFP Share']
data['Capital Deeping'] = data['Growth Rate'] - data['TFPgrowth'] 

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'Growth Rate': 'mean',
                                        'TFPgrowth':'mean',
                                        'Capital Deeping':'mean',
                                        'TFP Share':'mean',
                                       'Capital Share': "mean",
                                       })

summary.loc['Average'] = np.nan
summary.loc['Average',:]= summary.mean()

# Print output
print(summary)

             Growth Rate  TFPgrowth  Capital Deeping  TFP Share  Capital Share
countrycode                                                                   
CAN             0.887700   0.580760         0.306940   0.653810       0.346190
DEU             0.661055   0.411357         0.249698   0.629270       0.370730
FRA             0.829236   0.512309         0.316927   0.618806       0.381194
GBR             1.015544   0.588315         0.427229   0.587007       0.412993
ITA            -0.037181  -0.019589        -0.017592   0.515165       0.484835
JPN             0.710956   0.405471         0.305485   0.569399       0.430601
USA             1.536985   0.939222         0.597763   0.606740       0.393260
Average         0.800614   0.488263         0.312350   0.597171       0.402829
